In [7]:
#!pip install langdetect
import nltk
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/emilykim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilykim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import string
import re

from collections import Counter
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk import ngrams
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

# handle different languages
from langdetect import detect
#from googletrans import Translator

[nltk_data] Downloading package punkt to /Users/emilykim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilykim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# read combined and cleaned dataset
url = "https://raw.githubusercontent.com/AliceLiu17/csc448_final/main/data/combined_data_clean.csv"
df = pd.read_csv(url)

df.head()

,label,email
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
# check for NULL values in dataset 
df.isnull().sum()

label    0
email    0
dtype: int64

In [12]:
df.shape

(8029, 2)

In [15]:
# taken from Alice's notebook: alice/EDA_preprocessing_Alice.ipynb

df['email'] = df['email'].astype(str)

# function to determine the language of some of the text
# if it is gibberish, it'll return not available (N/A)
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'N/A'

df['language'] = df['email'].apply(detect_language)
non_english_rows = df[df['language'] != 'en'] # non-english text rows
non_english_labels = non_english_rows['label'] # non-english text labels

non_english_rows_count = df[df['language'] != 'en'].shape[0]
print("Number of non-english emails:", non_english_rows_count) # 508

Number of non-english emails: 508


In [16]:
# taken from Alice's notebook: alice/EDA_preprocessing_Alice.ipynb

# want to just have english text
# drop any non-english text
df_en = df[df['language'] == 'en']
df_en = df_en.drop(columns=['language'])
df_en.head()

,label,email
0,0,"Go until jurong point, crazy.. Available only ..."
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...


In [17]:
df_en.shape

(7522, 2)

In [22]:
# taken from my notebook (Emily): emily/data_analysis.ipynb

# create function that preprocesses the data
def preprocess_text(text):
    # handles 1. lowering text to keep it constant
    text = text.lower()
    
    # handles 2. breaking down the words in the emails
    tokens = word_tokenize(text)
    
    # handle 3. removing common words(stop words)
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    filtered_tokens = [token for token in tokens if token not in stop_words and token not in punctuation]
    
    return filtered_tokens

# adding new column to .csv dataset with processed data
df_en['processed_email'] = df_en['email'].apply(preprocess_text)

print(df_en['processed_email'].head())

0    [go, jurong, point, crazy, .., available, bugi...
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3    [u, dun, say, early, hor, ..., u, c, already, ...
4    [nah, n't, think, goes, usf, lives, around, th...
5    [freemsg, hey, darling, 's, 3, week, 's, word,...
Name: processed_email, dtype: object


In [23]:
df_en.head()

,label,email,processed_email
0,0,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, .., available, bugi..."
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, ..., u, c, already, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","[nah, n't, think, goes, usf, lives, around, th..."
5,1,FreeMsg Hey there darling it's been 3 week's n...,"[freemsg, hey, darling, 's, 3, week, 's, word,..."


In [25]:
# saving dataset to a csv file 
df_en.to_csv("preprocessed_english.csv", index = False)